In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from azure.identity import DefaultAzureCredential,get_bearer_token_provider
from langchain_openai import AzureChatOpenAI
from langgraph.graph import StateGraph,MessagesState,START,END
from langgraph.prebuilt import ToolNode,tools_condition
from IPython.display import display,Image
from langchain_core.messages import HumanMessage,AIMessage
import yfinance as yf
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnableConfig

@tool(return_direct=False)
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool(return_direct=True)
def greet(user_name: str,config: RunnableConfig) -> int:
    """Greet user."""
    user_id = config["configurable"].get("user_id")
    return f"Hello {user_name}! Your User ID is {user_id}"

@tool(return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    if a == 42:
        raise ValueError("The ultimate error")
    return a * b


token_provider= get_bearer_token_provider(DefaultAzureCredential(),"https://cognitiveservices.azure.com/.default")
chatmodel = AzureChatOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://azopenai-langchain.openai.azure.com/",
    azure_ad_token_provider= token_provider,
    model= "gpt-4o-mini"
)
tools=[add,greet,multiply]
agent = create_react_agent(
    # disable parallel tool calls
    model=chatmodel.bind_tools(tools, parallel_tool_calls=False,tool_choice="auto"),
    tools=tools
)

agent.invoke(
    {"messages": [{"role": "user", "content": "hi, I'm Vinod. how are you?"}]},
    config={"configurable": {"user_id": "user_123"}}
)

{'messages': [HumanMessage(content="hi, I'm Vinod. how are you?", additional_kwargs={}, response_metadata={}, id='fb6c02c5-17b8-41e1-a728-4313a453ce69'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ra0yT0lH91L3KGH0iqbaoW3w', 'function': {'arguments': '{"user_name":"Vinod"}', 'name': 'greet'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_57db37749c', 'id': 'chatcmpl-BjqZAXxA0Ch5yGAFTaNRk03dEdzzP', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_h

: 